In [ ]:
import os
from dotenv import load_dotenv

from autoLeague.dataset.generator import DataGenerator
from autoLeague.utils.csv_utils import save_to_csv
from autoLeague.utils.patch_info import get_patch_info

load_dotenv()  
API_KEY = os.environ["API_KEY"]
print(f"API_KEY: {API_KEY}")
get_patch_info()

<div style="font-size: 13px">
  <b>✅ 출력 예시 / Output example</b><br><br>
  <b>현재 시각 | Current time</b> : <code>2025-07-06 01:29:48</code><br>
  <b>과거 최신 패치 | Most recent past patch</b> : <code>25.13</code><br>
  <b>패치일 | Patch date</b> : <code>2025-06-25</code>

  <hr>

  <b>📝 Note</b><br><br>
  - 리플레이 <code>.rofl</code> 파일은 <b>현 패치 버전</b>에서만 다운로드 가능하므로, <b>패치 일정</b>을 확인한 뒤 아래 변수에 입력하세요.<br>
  - 최신 패치일 ~ 오늘 사이 아무 날짜를 넣어도 정상 동작합니다.


<pre><code>patch_start_datetime = "2025.06.30"  # → 2025-06-30 부터 오늘까지 경기 다운로드
</code></pre>

</div>

<div style="font-size: 13px">
  <b>✅ Output example</b><br><br>
  <b>Current time</b> : <code>2025-07-06 01:29:48</code><br>
  <b>Most recent past patch</b> : <code>25.13</code><br>
  <b>Patch date</b> : <code>2025-06-25</code>

  <hr>

  📝 <b>Note</b><br>
  - Replay <code>.rofl</code> files can be downloaded <b>only for the current patch version</b>, so check the patch schedule first and enter the date in the variable below.<br>
  - Any date between the latest patch date and today will work.


<pre style="font-size: 13px"><code>patch_start_datetime = "2025.06.30"  # → downloads matches from 2025-06-30 up to today
</code></pre>

</div>

<div style="font-size: 13px">
    <b> [KR] Data Generator 초기 세팅 : Riot API KEY 와, 각 소환사마다 최대 몇개의 경기를 불러올 것인지 세팅(api에서 최대 100까지 지원)</b></br><br>
    <b> [EN] Initial Data Generator Setup: Configure the "Riot API KEY" and the maximum number of matches to retrieve per summoner(~100 matches per summoner)</b></br><br>
</div>

In [5]:
dg = DataGenerator(api_key=API_KEY, count=100)

<div style="font-size: 13px">
    <b> [KR] </b></br></br>
    <b> [OPTION-1] : 하나의 특정 TIER, DIVISION 전적 조회 및 저장 </b></br>
    <b>            각 티어, 분기에서의 소환사들 전적(matchid) 조회 및 저장(csv)</b></br>
</br>
    <b> [EN] </b></br></br>
    <b> [OPTION-1] : Retrieve and save match history (match IDs) for a specific TIER and DIVISION. </b></br>
    <b>            Retrieve and save match histories (match IDs) of summoners for each tier and division as CSV files.</b></br></br>
</div>

In [6]:
matchIds_challenger = dg.get_tier_matchIds(
                                            queue='RANKED_SOLO_5x5',         # Queue type: Summoner's Rift Ranked Solo 5v5
                                            tier='CHALLENGER',               # Tier: CHALLENGER (highest competitive tier)
                                            division='I',                    # Division: I (CHALLENGER~MASTER technically only have one division "I" )
                                            max_ids=300,                     # Maximum number of match IDs to fetch
                                            patch_start_datetime='2025.06.26',  # Only fetch matches after this patch release date
                                            min_game_duration=15,           # Minimum game duration in minutes (filters out early surrenders/remakes)
                                            max_game_duration=45,            # Maximum game duration in minutes (filters out abnormal long matches)
                                                                            # With this setting, solo ranked matches in the CHALLENGER tier on the KR server from June 26 onward will be collected, limited to games lasting between 15 and 30 minutes.
                                                                            # The maximum number of summoners to collect is set to 300. (This is a reasonable number for the CHALLENGER tier, as it is the highest competitive tier and has fewer players than lower tiers.)
                                           max_workers=16
                                           )

save_to_csv(matchIds_challenger, rf'matchids/sample_challenger_patch25_13_1.csv', 'matchid')

[CHALLENGER-I] fetch matchIds: 100%|==================================================| 300/300 [00:41<00:00,  7.27it/s]


<div style="font-size: 13px">
    <b> [KR] </b></br><br>
    <b> [OPTION-2] : 여러 TIER, DIVISION 전적 조회 및 저장 </b></br>
    <b>            여러개 티어, 분기에서의 소환사들 전적(matchid) 조회 및 저장(csv)</b></br>
</br>
    <b> [EN] </b></br><br>
    <b> [OPTION-2] : Retrieve and save match histories across multiple TIERS and DIVISIONS. </b></br>
    <b>            Retrieve and save match histories (match IDs) of summoners across multiple TIERS and DIVISIONS as CSV files.</b></br>
</div>

In [6]:
# ── 설정값만 위에서 한 번 정의 ──────────────────────────────
PATCH_START     = "2025.06.26"                              # patch_start_datetime 값
PATCH_VER       = "25_13"                                   # 파일명에 넣을 패치 버전
TIERS           = ["CHALLENGER", "GRANDMASTER", "MASTER"]   # ["DIAMOND", "EMERALD", "PLATINUM", "GOLD", "SILVER", "BRONZE", "IRON"]
MAXIDS_PER_TIER = [300, 700, 1000]                          # 각 티어별로 최대 몇 개의 matchid 를 가져올지
DIVISIONS       = ["I"]                                     # 다이아몬드 이하에서만 I ~ IV 적용 ["I", "II", "III", "IV"]
OUT_DIR         = "matchids"                                # 저장 경로 (예 : pyLoL/matchids)

# ── 폴더 준비 ───────────────────────────────────────────────
import os, pathlib
pathlib.Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

# ── 다운로드 + CSV 저장 루프 ────────────────────────────────
for tier, max_ids in zip(TIERS, MAXIDS_PER_TIER):
    for div in DIVISIONS:
        match_ids = dg.get_tier_matchIds(
            queue="RANKED_SOLO_5x5",
            tier=tier,
            division=div,
            max_ids=max_ids,
            patch_start_datetime=PATCH_START,
            min_game_duration=15,
            max_game_duration=60,
            max_workers=8
        )
        save_to_csv(
            match_ids,
            rf"{OUT_DIR}/matchids_{tier}_{div}_patch{PATCH_VER}.csv",
            "matchid",
        )

[MASTER-I] fetch matchIds: 100%|====================================================| 1000/1000 [13:04<00:00,  1.28it/s]
